In [27]:
from google.colab import drive
import nbformat, os

drive.mount('/content/drive', force_remount=True)
SRC = '/content/drive/MyDrive/Colab Notebooks/Untitled11.ipynb'
DST = '/content/drive/MyDrive/colab_saves/tune last 3 layers.ipynb'

with open(SRC, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)


nb.metadata.pop('widgets', None)
for cell in nb.cells:
    cell.metadata.pop('widgets', None)
os.makedirs(os.path.dirname(DST), exist_ok=True)
with open(DST, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("✅ Clean copy saved to", DST)


Mounted at /content/drive
✅ Clean copy saved to /content/drive/MyDrive/colab_saves/tune last 3 layers.ipynb


In [1]:
!pip install datasets sentence-transformers
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [2]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample
import torch
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from datasets import load_dataset
from torch.utils.data import DataLoader
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import numpy as np

In [3]:
# call English STS-Benchmark
from datasets import load_dataset
raw = load_dataset("glue", "stsb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [4]:
from sentence_transformers import InputExample

def to_examples(split):
    examples = []
    for row in raw[split]:
        examples.append(
            InputExample(
                texts=[row["sentence1"], row["sentence2"]],
                label=float(row["label"])))
    return examples

train_examples = to_examples("train")
dev_examples   = to_examples("validation")



In [5]:


model = SentenceTransformer("sentence-transformers/stsb-roberta-large")
for param in model.parameters():
    param.requires_grad = False

transformer_module = model._modules['0']

for param in transformer_module.auto_model.embeddings.parameters():
    param.requires_grad = True

for layer in transformer_module.auto_model.encoder.layer[-3:]:
    for param in layer.parameters():
        param.requires_grad = True

train_dataset    = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss       = CosineSimilarityLoss(model)

dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_examples, name="sts-dev")




modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"



In [7]:


best_spearman = -1.0
no_improve    = 0
patience      = 5

for epoch in range(1, 11):
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        evaluation_steps=0,
        save_best_model=True,
        output_path="output/sts-3layers",
        evaluator=dev_evaluator,)

    results = dev_evaluator(model, output_path=None, epoch=epoch, steps=0)

    spearman_key = next(k for k in results if "spearman" in k)
    curr = results[spearman_key]

    if curr > best_spearman + 1e-4:
        best_spearman, no_improve = curr, 0
    else:
        no_improve += 1

    if no_improve >= patience:
        print(f"⏹ Early stopping at epoch {epoch}, best Spearman={best_spearman:.4f}")
        break


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.869952,0.875443


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.844041,0.858365


...

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.767200,0.817205


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.767200,0.817205


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.769668,0.818961


⏹ Early stopping at epoch 6, best Spearman=0.8754


In [8]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
MODEL_DIR = '/content/drive/MyDrive/colab_saves/sts-3layers'

os.makedirs(MODEL_DIR, exist_ok=True)

model.save(MODEL_DIR)

print(f">>> model saved to {MODEL_DIR}")


Mounted at /content/drive
>>> model saved to /content/drive/MyDrive/colab_saves/sts-3layers


In [9]:
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

API_KEY = os.getenv("RAPIDAPI_KEY") or "ad973560c8msh96909b7fb3cc3fdp1c1065jsn5a49e6b93474"
HEADERS = {
    "X-RapidAPI-Key":  API_KEY,
    "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com",
}

session = requests.Session()
retry_cfg = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"],
)
session.mount("https://", HTTPAdapter(max_retries=retry_cfg))


def _respect_rate_limit(resp):
    """Sleep if remaining-requests header is low."""
    left = resp.headers.get("X-RateLimit-Requests-Remaining")
    reset = resp.headers.get("X-RateLimit-Requests-Reset")
    if left is not None and reset is not None:
        left, reset = int(left), int(reset)
        if left < 2:
            sleep_s = max(reset - int(time.time()), 1)
            print(f"[rate-limit] only {left} calls left → sleeping {sleep_s}s")
            time.sleep(sleep_s)

# extracting the news with particular symbols

def fetch_symbol_news(symbol: str,
                      max_items: int = 200,
                      page_size: int = 40,
                      since: int = 0,
                      until: int = 0) -> pd.DataFrame:
    url   = "https://seeking-alpha.p.rapidapi.com/news/v2/list-by-symbol"
    items = []
    page  = 1

    while len(items) < max_items:
        p = {"id": symbol, "size": page_size, "number": page}
        if since: p["since"] = since
        if until: p["until"] = until

        r = session.get(url, headers=HEADERS, params=p, timeout=20)
        r.raise_for_status()
        batch = r.json().get("data", [])
        if not batch:
            break

        items.extend(batch)
        if len(batch) < page_size:
            break

        page += 1
        time.sleep(0.25 + 0.25 * os.urandom(1)[0] / 255)

    if not items:
        return pd.DataFrame(columns=["title", "publishOn", "symbol"])

    df = pd.json_normalize(items[:max_items])

    for c in ["attributes.title", "attributes.headline", "attributes.teaser"]:
        if c in df.columns:
            df = df.rename(columns={c: "title"})
            break
    if "title" not in df.columns:
        df["title"] = ""

    if "publishOn" in df.columns:
        df["publishOn"] = pd.to_datetime(df["publishOn"], utc=True)
    elif "publishedAt" in df.columns:
        df["publishOn"] = pd.to_datetime(df["publishedAt"], utc=True)

    df["symbol"] = symbol
    return df



In [10]:
# collect the return data on the publish day of news and convert to 1,0
import yfinance as yf
def label_news_df(df: pd.DataFrame, symbol: str) -> pd.DataFrame:
    df = df.copy()
    if "attributes.publishOn" in df.columns:
        df = df.rename(columns={"attributes.publishOn": "publishOn"})


    df["publishOn"] = (
        pd.to_datetime(df["publishOn"], utc=True, errors="coerce")
          .dt.tz_localize(None)
    )
    df = df.dropna(subset=["publishOn"])

    # 1) build price window
    earliest = df["publishOn"].dt.date.min()
    latest   = df["publishOn"].dt.date.max()
    start_dt = (pd.Timestamp(earliest) - pd.Timedelta(days=7)).strftime("%Y-%m-%d")
    end_dt   = (pd.Timestamp(latest)   + pd.Timedelta(days=7)).strftime("%Y-%m-%d")

    px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")

    # extract the Close price
    closes = px["Close"]
    if isinstance(closes, pd.DataFrame):
        closes = closes.squeeze()

    # normalise to midnight dates
    closes.index = closes.index.tz_localize(None).normalize()
    trading_dates = closes.index

    ret_vals, labels = [], []
    for ts in df["publishOn"]:
        pub = pd.to_datetime(ts).tz_localize(None).normalize()

        prev_days = trading_dates[trading_dates < pub]
        next_days = trading_dates[trading_dates > pub]
        if prev_days.empty or next_days.empty:
            ret_vals.append(np.nan)
            labels.append(np.nan)
            continue

        d0, d1 = prev_days.max(), next_days.min()
        p0, p1 = closes.at[d0], closes.at[d1]

        if pd.isna(p0) or pd.isna(p1) or p0 == 0:
            ret_vals.append(np.nan)
            labels.append(np.nan)
            continue

        r = p1 / p0 - 1
        ret_vals.append(r)
        labels.append(int(r > 0))

    df["return_1d"] = ret_vals
    df["label"]     = labels
    df = df.dropna(subset=["label"]).reset_index(drop=True)
    return df




In [11]:
from google.colab import drive
import pickle, os

drive.mount('/content/drive', force_remount=True)

path = '/content/drive/MyDrive/colab_saves/company_dfs.pkl'

!ls -lh "{path}"
with open(path, 'rb') as f:
    company_dfs = pickle.load(f)
print(type(company_dfs))
print(company_dfs.keys())

Mounted at /content/drive
-rw------- 1 root root 14M Jul 16 10:03 /content/drive/MyDrive/colab_saves/company_dfs.pkl
<class 'dict'>
dict_keys(['JPM', 'BAC', 'WFC', 'RY', 'group2', 'group3', 'group4'])


In [12]:


symbols_2 = ["C", "GS", "MS", "USB"]

company_df_2 = {
    sym: fetch_symbol_news(sym, max_items=400, since=0, until=0)
    for sym in symbols_2}



KeyboardInterrupt: 

In [ ]:


symbols_3 = ["AAPL", "MSFT", "AMZN", "GOOGL"]

company_df_3 = {
    sym: fetch_symbol_news(sym, max_items=400, since=0, until=0)
    for sym in symbols_3}



In [ ]:

symbols_4 = ["META", "NVDA", "TSLA", "NFLX"]

company_df_4 = {
    sym: fetch_symbol_news(sym, max_items=400, since=0, until=0)
    for sym in symbols_4}



In [14]:
import pickle

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = '/content/drive/My Drive/colab_saves/company_dfs.pkl'
with open(path, 'rb') as f:
    company_dfs = pickle.load(f)

company_df_2 = company_dfs['group2']
company_df_3 = company_dfs['group3']
company_df_4 = company_dfs['group4']


Mounted at /content/drive


In [15]:

import numpy as np

labeled_group2 = {
    sym: label_news_df(df, sym)
    for sym, df in company_df_2.items()
}

labeled_group3 = {
    sym: label_news_df(df, sym)
    for sym, df in company_df_3.items()
}

labeled_group4 = {
    sym: label_news_df(df, sym)
    for sym, df in company_df_4.items()
}



/tmp/ipython-input-10-599390206.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-10-599390206.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-10-599390206.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-10-599390206.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")
[*********************100%*******************

In [26]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

results = {}

for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"=== Processing {group_name} ===")

    df_all    = pd.concat(labeled_group.values(), ignore_index=True)
    sentences = df_all["title"].tolist()
    y_all     = df_all["label"].astype(int).to_numpy()
    X_all     = model.encode(sentences, convert_to_tensor=False)

    X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
        X_all, y_all,
        test_size=0.2,
        random_state=SEED,
        stratify=y_all)

    X_train = torch.tensor(X_train_np, dtype=torch.float32)
    y_train = torch.tensor(y_train_np, dtype=torch.long)
    X_test  = torch.tensor(X_test_np,  dtype=torch.float32)
    y_test  = torch.tensor(y_test_np,  dtype=torch.long)

    train_ds = TensorDataset(X_train, y_train)
    train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

    head      = nn.Linear(X_train.size(1), 2)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(head.parameters(), lr=1e-3)

    for epoch in range(1, 6):
        head.train()
        total_loss = 0.0
        for emb, lab in train_dl:
            logits = head(emb)
            loss   = criterion(logits, lab)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item() * emb.size(0)
        print(f"  Epoch {epoch:>2} — Train loss: {total_loss / len(train_ds):.4f}")

    head.eval()
    with torch.no_grad():
        logits_test = head(X_test)
        preds_test  = logits_test.argmax(dim=1).cpu().numpy()
        y_true_test = y_test.cpu().numpy()

    acc          = accuracy_score(y_true_test, preds_test)
    micro_f1     = f1_score(y_true_test, preds_test, average="micro")
    macro_f1     = f1_score(y_true_test, preds_test, average="macro")
    weighted_f1  = f1_score(y_true_test, preds_test, average="weighted")
    macro_recall = recall_score(y_true_test, preds_test, average="macro")

    print(f"\nTest-set metrics for {group_name}:")
    print(f"  Accuracy      : {acc*100:6.2f}%")
    print(f"  Micro-F1      : {micro_f1*100:6.2f}%")
    print(f"  Macro-F1      : {macro_f1*100:6.2f}%")
    print(f"  Weighted-F1   : {weighted_f1*100:6.2f}%")
    print(f"  Macro-Recall  : {macro_recall*100:6.2f}%\n")

    results[group_name] = head


=== Processing group2 ===
  Epoch  1 — Train loss: 0.6939
  Epoch  2 — Train loss: 0.6744
  Epoch  3 — Train loss: 0.6670
  Epoch  4 — Train loss: 0.6598
  Epoch  5 — Train loss: 0.6596

Test-set metrics for group2:
  Accuracy      :  54.37%
  Micro-F1      :  54.37%
  Macro-F1      :  39.17%
  Weighted-F1   :  41.64%
  Macro-Recall  :  50.65%

=== Processing group3 ===
  Epoch  1 — Train loss: 0.7009
  Epoch  2 — Train loss: 0.6795
  Epoch  3 — Train loss: 0.6716
  Epoch  4 — Train loss: 0.6653
  Epoch  5 — Train loss: 0.6622

Test-set metrics for group3:
  Accuracy      :  53.75%
  Micro-F1      :  53.75%
  Macro-F1      :  51.70%
  Weighted-F1   :  52.94%
  Macro-Recall  :  51.98%

=== Processing group4 ===
  Epoch  1 — Train loss: 0.6975
  Epoch  2 — Train loss: 0.6804
  Epoch  3 — Train loss: 0.6688
  Epoch  4 — Train loss: 0.6674
  Epoch  5 — Train loss: 0.6615

Test-set metrics for group4:
  Accuracy      :  55.94%
  Micro-F1      :  55.94%
  Macro-F1      :  54.49%
  Weighted-F